# Compare explaners

In this notebook, we investigate and measure the plausibility of other explainers :
* LIME: done
* SHAP: done
* Gradient-based: done
* DeepLIFT: TODO
* LRP: need implementation for each layers
* Rationalized learning : TODO - to see if can use another framework/model to train for this task

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
import sys
import os
from os import path

sys.path.append("./../src")

In [2]:
from modules.logger import init_logging
from modules.logger import log

init_logging(color=True)

In [3]:
!nvidia-smi

Fri Oct  6 05:24:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:04:00.0 Off |                  N/A |
| 23%   22C    P8     8W / 250W |      1MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torchmetrics as m
import warnings

# Measure plausibility with AUPRC
warnings.filterwarnings('ignore')
auprc = m.AveragePrecision(average='micro')
warnings.filterwarnings('default')

# LIME

## HateXplain

In [5]:
import platform

node = platform.node()

print('Current node:', node)

# log to lstm models
DATASET = 'hatexplain'
MODEL_VERSION = 'run=0_lstm=1'

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else: 
    ROOT = '/home/dunguyen/RUNS'
    
LOG_PATH = path.join(ROOT, 'logs')
DATA_CACHE = path.join(ROOT, 'dataset')


MODEL_PATH = path.join(ROOT, 'logs','lstm_attention',DATASET, MODEL_VERSION)


Current node: grele-3.nancy.grid5000.fr


In [6]:
from data_module.hatexplain_module import HateXPlainDM
from model_module import SingleLSTMAttentionModule
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

###############
# PREPARE DATA
###############
hatexplain_dm = HateXPlainDM(cache_path=DATA_CACHE, batch_size=8, fetch_data=True)
hatexplain_dm.prepare_data()
hatexplain_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=1,
    #concat_context=True,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = SingleLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=hatexplain_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(hatexplain_dm.LABEL_ITOS), 
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)
    
model.to(device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

04-10-2023 13:47:40 |     INFO   hatexplain_module.py:prepare_data:76  Loaded vocab at /home/dunguyen/RUNS/dataset/hatexplain/vocab.pt
04-10-2023 13:47:40 |     INFO   hatexplain_module.py:prepare_data:78  Vocab size: 24993
04-10-2023 13:47:50 |    DEBUG   single_lstm_attention.py:__init__:35  Initialize embedding from pretrained vector
04-10-2023 13:47:50 |    DEBUG   single_lstm_attention.py:__init__:57  concat_context: True


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


In [7]:
from modules.const import SpecToken
from lime.lime_text import LimeTextExplainer

lime_explainer = LimeTextExplainer(
    class_names=hatexplain_dm.LABEL_ITOS,
    verbose=False
)

In [8]:
def inference_lstm(inputs):
    # tokenize by space
    batch_tokens = [txt.split(' ') for txt in inputs]
    token_ids = hatexplain_dm.text_transform(batch_tokens).to(device)
    with torch.no_grad():
        y_hat = model(token_ids)

    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

def inference_attention(inputs):
    # tokenize by space
    batch_tokens = [txt.split(' ') for txt in inputs]
    token_ids = hatexplain_dm.text_transform(batch_tokens).to(device)
    padding_mask = token_ids == hatexplain_dm.vocab[SpecToken.PAD]
    with torch.no_grad():
        y_hat, attention = model(ids=token_ids, mask=padding_mask)

    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

In [12]:
# Get one data instance
idx = 100
post_tokens, label = hatexplain_dm.test_set.data.loc[idx, ['post_tokens', 'label']]
text = ' '.join(post_tokens)
print('Text: ', text)
print('Label: ', label)

# Predicting on data
token_ids = hatexplain_dm.text_transform([post_tokens]).to(device)
padding_mask = token_ids == hatexplain_dm.vocab[SpecToken.PAD]
y_hat, attention = model(token_ids, padding_mask)
y_hat = y_hat.argmax(-1).item()
label_hat = hatexplain_dm.LABEL_ITOS[y_hat]
print('Prediction: ', label_hat)

# explain using LIME
lime_exp = lime_explainer.explain_instance(text, inference_attention, num_features = 7, num_samples=28, labels=[y_hat])
lime_exp = dict(lime_exp.as_map()[y_hat])
lime_map = [lime_exp.get(i, 0) for i in range(len(text.split(' ')))]
lime_map = torch.tensor(lime_map)
lime_map[lime_map < 0] = 0

# Measure AUPRC against annotation
a_true = hatexplain_dm.test_set.data.loc[idx, 'rationale']
a_true = torch.tensor(a_true)
auprc.update(lime_map, a_true)
print('LIME: ', lime_map)
print('Rationale: ', a_true)
print('AURPC : ', auprc.compute())

# reset metrics to avoid cumulation
auprc.reset()

Text:  backpedaling fails to point out exactly my comments and resumes to nonsensical aussie retarded babbles go choke on a cock you useless wrinkly aussie slag you are th descendants of the bottom feeders among limeys loll
Label:  hatespeech
Prediction:  offensive
LIME:  tensor([0.0000, 0.0000, 0.0201, 0.0326, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0782, 0.0000, 0.0000, 0.0000, 0.0147, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0241, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       dtype=torch.float64)
Rationale:  tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
AURPC :  tensor(0.8004)


In [9]:
%%time
import pandas as pd
from tqdm.auto import tqdm
from modules import rescale
import numpy as np
import scipy.stats as st

cumulative_auprc = []
#NUM_RUN = 3
NUM_RUN = 1
WRITE_OUTPUT = True
output_data = []

# Rerun this computation NUM_RUN times to get the average AUPRC
for idx_run in range(NUM_RUN):

    n_zero = 0
    for batch in tqdm(hatexplain_dm.test_dataloader(), total=len(hatexplain_dm.test_set)//hatexplain_dm.batch_size):

        with torch.no_grad():
            # y_hat = model(batch['tokens'].to(device))
            token_ids = batch['tokens.ids'].to(device)
            padding_mask = token_ids == hatexplain_dm.vocab[SpecToken.PAD]
            y_hat, attention = model(token_ids, padding_mask)
        y_hat = y_hat.argmax(-1).cpu()
        y_true = batch['y_true']

        a_hat = []
        a_true = []

        true_positive_idx = (y_hat == y_true) & (y_true > 0)

        if true_positive_idx.sum() == 0 and not WRITE_OUTPUT:
            n_zero += 1
            continue

        # Iterate each line in batch
        for i in range(len(y_hat)):

            label_idx = batch['y_true'][i]

            if (y_hat[i] != label_idx or label_idx == 0) and not WRITE_OUTPUT:
                continue

            text_tokens = batch['post_tokens'][i]
            num_feature = len(text_tokens)
            text_tokens = ' '.join(text_tokens)
            lime_exp = lime_explainer.explain_instance(text_tokens, inference_attention, num_features = num_feature, num_samples=512, labels=[label_idx])

            lime_exp = dict(lime_exp.as_map()[label_idx])
            lime_map = [lime_exp.get(i, 0) for i in range(len(text_tokens.split(' ')))]

            lime_map = torch.tensor(lime_map)
            lime_map[lime_map < 0] = 0
            lime_map = rescale(lime_map)

            a_hat.append(lime_map)
            rationale = torch.tensor(batch['rationale'][i])
            a_true.append(rationale)

            # Write output
            if WRITE_OUTPUT:
                output_data.append({
                    'id': batch['id'][i],
                    'label': hatexplain_dm.LABEL_ITOS[label_idx],
                    'tokens.norm': batch['tokens.norm'][i],
                    'a_lime': lime_map.tolist()
                })

        a_hat = torch.cat(a_hat, dim=0)
        a_true = torch.cat(a_true, dim=0)
        auprc.update(a_hat, a_true)

    auprc_value = auprc.compute().item()
    cumulative_auprc.append(auprc_value)
    print('AURPC', auprc_value)
    print('n_zero', n_zero)
    auprc.reset()

    # Write output
    if WRITE_OUTPUT:
        df_lime = pd.DataFrame(output_data)
        df_lime.to_json(path.join(MODEL_PATH, 'predictions', 'lime_map.json'))

# Compute confidential interval
ci_auprc = np.array(cumulative_auprc)
if len(ci_auprc) == 3:
    print('CI AUPRC:', str(round(ci_auprc.mean(), 3)) , ' ± ' , str(round(1.96 * st.sem(ci_auprc), 3)))
else:
    print(ci_auprc[0])

  0%|          | 0/240 [00:00<?, ?it/s]

AURPC 0.25795698165893555
n_zero 0
0.25795698165893555
CPU times: user 23min 53s, sys: 25min 9s, total: 49min 3s
Wall time: 2min 45s


In [24]:
from modules import highlight

true_positive_idx = (y_hat == y_true) & (y_true > 0)

post_tokens = [tokens for true_positive, tokens in zip(true_positive_idx, batch['post_tokens']) if true_positive]

for lmap, txt, y_true_ins, y_hat_ins in zip(lime_map, post_tokens, y_true[true_positive_idx], y_hat[true_positive_idx]):
    
    display(HTML(highlight(txt, lmap)))
    print('y_true :', hatexplain_dm.LABEL_ITOS[y_true_ins], '| y_hat :', hatexplain_dm.LABEL_ITOS[y_hat_ins])
    print('='*12)

y_true : offensive | y_hat : offensive


y_true : hatespeech | y_hat : hatespeech


In [65]:
from modules import metrics

entropy = metrics.Entropy(normalize=True)
n_a_true = torch.tensor([])

with torch.no_grad():
    for batch in hatexplain_dm.test_dataloader():
        entropy.update(batch['a_true'][batch['y_true'] > 0])
        n_a_true_ins = batch['a_true'][batch['y_true'] > 0].sum(-1)
        n_a_true = torch.cat([n_a_true, n_a_true_ins])
        
print('entropy', entropy.compute())
entropy.reset()

entropy tensor(0.9839)


In [68]:
n_a_true.numpy().mean()

7.8021016

## YelpHat

In [10]:
import platform; 

node = platform.node()

print('Current node:', node)

# log to lstm models
DATASET = 'yelphat50'
MODEL_VERSION = 'run=0_lstm=1'

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else: 
    ROOT = '/home/dunguyen/RUNS'
    
LOG_PATH = path.join(ROOT, 'logs')
DATA_CACHE = path.join(ROOT, 'dataset_')

MODEL_PATH = path.join(ROOT, 'logs','lstm_attention',DATASET, MODEL_VERSION)

Current node: grele-3.nancy.grid5000.fr


In [ ]:
from data_module.yelp_hat_module import YelpHat50DM
from model_module import SingleLSTMAttentionModule
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

###############
# PREPARE DATA
###############
yelphat_dm = YelpHat50DM(cache_path=DATA_CACHE, batch_size=16)
yelphat_dm.prepare_data()
yelphat_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=1,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = SingleLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=yelphat_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(yelphat_dm.LABEL_ITOS), 
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)
    
model.to(device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

04-10-2023 14:41:32 |     INFO   yelp_hat_module.py:prepare_data:89  Loaded vocab at /home/dunguyen/RUNS/dataset_/yelp-hat/vocab_lemma_lower.pt
04-10-2023 14:41:32 |     INFO   yelp_hat_module.py:prepare_data:91  Vocab size: 7465
04-10-2023 14:41:32 |     INFO   dataset.py:__init__:186  Load dataset from /home/dunguyen/RUNS/dataset_/yelp-hat/yelp50.parquet


In [7]:
from lime.lime_text import LimeTextExplainer

lime_explainer = LimeTextExplainer(
    class_names=yelphat_dm.LABEL_ITOS,
    verbose=False
)

def inference_lstm(inputs):
    # tokenize by space
    batch_tokens = [txt.split(' ') for txt in inputs]
    token_ids = yelphat_dm.text_transform(batch_tokens).to(device)
    with torch.no_grad():
        y_hat = model(token_ids)

    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

def inference_attention(inputs):
    # tokenize by space
    batch_tokens = [txt.split(' ') for txt in inputs]
    token_ids = yelphat_dm.text_transform(batch_tokens).to(device)
    padding_mask = token_ids == yelphat_dm.vocab[SpecToken.PAD]
    with torch.no_grad():
        y_hat, attention = model(ids=token_ids, mask=padding_mask)

    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

In [8]:
from modules.const import SpecToken
from lime.lime_text import LimeTextExplainer

lime_explainer = LimeTextExplainer(
    class_names=yelphat_dm.LABEL_ITOS,
    verbose=False
)

In [11]:
%%time 
from tqdm.auto import tqdm
from modules import rescale
import numpy as np
import scipy.stats as st
import pandas as pd

cumulative_auprc = []
#NUM_RUN = 3
NUM_RUN = 1

WRITE_OUTPUT = True
output_data = []

# Rerun this computation NUM_RUN times to get the average AUPRC
for idx_run in range(NUM_RUN):

    n_zero = 0
    for batch in tqdm(yelphat_dm.test_dataloader(), total=len(yelphat_dm.test_set)//yelphat_dm.batch_size):

        with torch.no_grad():
            token_ids = batch['tokens.ids'].to(device)
            padding_mask = batch['padding_mask'].to(device)
            y_hat, attention = model(token_ids, padding_mask)
            #y_hat = model(batch['tokens'].to(device))
        y_hat = y_hat.argmax(-1).cpu()
        y_true = batch['y_true']

        a_hat = []
        a_true = []

        true_positive_idx = (y_hat == y_true) & (y_true > 0)

        if true_positive_idx.sum() == 0:
            n_zero += 1
            continue

        # Iterate each line in batch
        for i in range(len(y_hat)):

            label_idx = batch['y_true'][i]

            if y_hat[i] != label_idx or label_idx == 0:
                continue

            text_tokens = batch['tokens.norm'][i]
            num_feature = len(text_tokens)
            text_tokens = ' '.join(text_tokens)
            lime_exp = lime_explainer.explain_instance(text_tokens, inference_attention, num_features = num_feature, num_samples=512, labels=[label_idx])

            lime_exp = dict(lime_exp.as_map()[label_idx])
            lime_map = [lime_exp.get(i, 0) for i in range(len(text_tokens.split(' ')))]

            lime_map = torch.tensor(lime_map)
            lime_map[lime_map < 0] = 0
            lime_map = rescale(lime_map)

            a_hat.append(lime_map)
            rationale = torch.tensor(batch['ham'][i])
            a_true.append(rationale)

            # Write output
            if WRITE_OUTPUT:
                output_data.append({
                    'id': batch['id'][i],
                    'label': yelphat_dm.LABEL_ITOS[label_idx],
                    'tokens.norm': batch['tokens.norm'][i],
                    'a_lime': lime_map.tolist()
                })

        a_hat = torch.cat(a_hat, dim=0)
        a_true = torch.cat(a_true, dim=0)
        #a_true = batch['a_true'][~batch['padding_mask']]

        auprc.update(lime_map, a_true)

    auprc_value = auprc.compute().item()
    cumulative_auprc.append(auprc_value)
    print('AURPC', auprc_value)
    print('n_zero', n_zero)
    auprc.reset()

    # Write output
    if WRITE_OUTPUT:
        df_lime = pd.DataFrame(output_data)
        df_lime.to_json(path.join(MODEL_PATH, 'predictions', 'lime_map.json'))

# Compute confidential interval
ci_auprc = np.array(cumulative_auprc)
print('CI AUPRC:', str(round(ci_auprc.mean(), 3)) , '±' , str(round(1.96 * st.sem(ci_auprc), 3)))

  0%|          | 0/8 [00:00<?, ?it/s]

AURPC 0.11733417212963104
n_zero 4
CI AUPRC: 0.117 ± nan
CPU times: user 1min 29s, sys: 1min 34s, total: 3min 4s
Wall time: 9.25 s


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [31]:
from modules import metrics

entropy = metrics.Entropy(normalize=True)
n_a_true = torch.tensor([])

with torch.no_grad():
    for batch in yelphat_dm.test_dataloader():
        entropy.update(batch['a_true'][batch['y_true'] > 0])
        n_a_true_ins = batch['a_true'][batch['y_true'] > 0].sum(-1)
        n_a_true = torch.cat([n_a_true, n_a_true_ins])
        

print('entropy', entropy.compute())
entropy.reset()

print('#Annot:' ,str(round(n_a_true.numpy().mean(), 3)),'±', str(round(1.96 * st.sem(n_a_true.numpy()), 3)))

entropy tensor(0.9774)
#Annot: 11.375 ± 1.44


## e-SNLI

In [5]:
import platform

node = platform.node()

print('Current node:', node)

# log to lstm models
DATASET = 'esnli'
MODEL_VERSION = 'run=0_lstm=1'

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
    # Check if the new dataset is appropriate
else: 
    ROOT = '/home/dunguyen/RUNS'
    
LOG_PATH = path.join(ROOT, 'logs')
DATA_CACHE = path.join(ROOT, 'dataset_')
#DATA_CACHE = path.join('./../.cache', 'dataset') # Check if the new dataset is appropriate


MODEL_PATH = path.join(ROOT, 'logs','lstm_attention',DATASET, MODEL_VERSION)


Current node: grele-2.nancy.grid5000.fr


In [6]:
from data_module.esnli_module import ESNLIDM
from model_module import DualLSTMAttentionModule
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

###############
# PREPARE DATA
###############
esnli_dm = ESNLIDM(cache_path=DATA_CACHE, batch_size=16)
esnli_dm.prepare_data()
esnli_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=1,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = DualLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=esnli_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(esnli_dm.LABEL_ITOS), 
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)
    
model.to(device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

03-10-2023 13:05:44 |     INFO   esnli_module.py:prepare_data:103  Loaded vocab at /home/dunguyen/RUNS/dataset_/esnli/vocab.pt
03-10-2023 13:05:44 |     INFO   esnli_module.py:prepare_data:105  Vocab size: 26578
03-10-2023 13:05:44 |  WARNING   dual_lstm_attention_module.py:__init__:53  Check if everything is ok here for archival !
03-10-2023 13:05:48 |    DEBUG   dual_lstm_attention.py:__init__:36  Initialize embedding from pretrained vector


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


In [7]:
from lime.lime_text import LimeTextExplainer

lime_explainer = LimeTextExplainer(
    class_names=esnli_dm.LABEL_ITOS,
    verbose=False
)

In [8]:
row = esnli_dm.test_set.data.loc[10, ['tokens.form.premise', 'tokens.form.hypothesis', 'label']]
premise_tokens = row['tokens.form.premise']
hypothesis_tokens = row['tokens.form.hypothesis']

texts = ' '.join(premise_tokens) + ' ' + ' '.join(hypothesis_tokens)
print(texts)
print(row['label'])

A statue at a museum that no seems to be looking at . There is a statue that not many people seem to be interested in .
entailment


In [9]:
def inference(inputs):
    """ Wrap model's predictions
    """
    batch_tokens = [txt.split(' ') for txt in inputs]
    
    premise_tokens = [tokens[:premise_len] for tokens in batch_tokens]
    hypothesis_tokens = [tokens[premise_len:] for tokens in batch_tokens]
    
    premise_ids = esnli_dm.text_transform(premise_tokens).to(device)
    hypothesis_ids = esnli_dm.text_transform(hypothesis_tokens).to(device)

    premise_padding =  premise_ids == esnli_dm.vocab[SpecToken.PAD]
    hypothesis_padding = hypothesis_ids == esnli_dm.vocab[SpecToken.PAD]
    
    with torch.no_grad():
        y_hat, a_hat = model(premise_ids, hypothesis_ids, premise_padding, hypothesis_padding)
        
    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

label_idx=[1,]
premise_len = len(premise_tokens)
lime_exp = lime_explainer.explain_instance(texts, inference, num_features = 4, num_samples=12, labels=label_idx)

lime_exp.show_in_notebook(text=texts, labels=label_idx)

NameError: name 'SpecToken' is not defined

In [14]:
%%time 
from tqdm.auto import tqdm
from modules import rescale
from functools import partial
import numpy as np
import scipy.stats as st
from modules.const import SpecToken
import pandas as pd

def inference_lstm(inputs, premise_len=None):
    """ Wrap model's predictions
    """
    assert premise_len is not None, 'premise_len must be specified'

    batch_tokens = [txt.split(' ') for txt in inputs]

    premise_tokens = [tokens[:premise_len] for tokens in batch_tokens]
    hypothesis_tokens = [tokens[premise_len:] for tokens in batch_tokens]

    premise_ids = esnli_dm.text_transform(premise_tokens).to(device)
    hypothesis_ids = esnli_dm.text_transform(hypothesis_tokens).to(device)

    with torch.no_grad():
        y_hat = model(premise_ids, hypothesis_ids)

    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

def inference_attention(inputs, premise_len=None):
    assert premise_len is not None, 'premise_len must be specified'

    # tokenize by space
    batch_tokens = [txt.split(' ') for txt in inputs]

    premise_tokens = [tokens[:premise_len] for tokens in batch_tokens]
    hypothesis_tokens = [tokens[premise_len:] for tokens in batch_tokens]

    premise_ids = esnli_dm.text_transform(premise_tokens).to(device)
    hypothesis_ids = esnli_dm.text_transform(hypothesis_tokens).to(device)

    premise_padding =  premise_ids == esnli_dm.vocab[SpecToken.PAD]
    hypothesis_padding = hypothesis_ids == esnli_dm.vocab[SpecToken.PAD]
    with torch.no_grad():
        y_hat, attention = model(
            premise_ids=premise_ids,
            hypothesis_ids=hypothesis_ids,
            premise_padding=premise_padding,
            hypothesis_padding=hypothesis_padding)

    y_hat = y_hat.softmax(-1)
    y_hat = y_hat.cpu().numpy()
    return y_hat

cumulative_auprc = []
# NUM_RUN = 3
NUM_RUN = 1
WRITE_OUTPUT = True
output_data = []

for idx_run in range(NUM_RUN):

    n_zero = 0
    n_inconsistency = 0
    pbar = tqdm(desc='LIME eSNLI', total=len(esnli_dm.test_set)//esnli_dm.batch_size))
    for batch in esnli_dm.test_dataloader():
        premise_ids = esnli_dm.text_transform(batch['tokens.norm.premise']).to(device)
        hypothesis_ids = esnli_dm.text_transform(batch['tokens.norm.hypothesis']).to(device)

        premise_padding =  premise_ids == esnli_dm.vocab[SpecToken.PAD]
        hypothesis_padding = hypothesis_ids == esnli_dm.vocab[SpecToken.PAD]

        with torch.no_grad():
            y_hat, attention = model(premise_ids, hypothesis_ids, premise_padding, hypothesis_padding)
        y_hat = y_hat.argmax(-1).cpu()
        y_true = batch['y_true']

        true_positive_idx = (y_hat == y_true) & (y_true > 0)

        # Get predicted premise and hypothesis rationales
        a_hat = []
        a_true = []

        if true_positive_idx.sum() == 0:
            n_zero += 1
            continue

        # Iterate each line in batch
        for i in range(len(y_hat)):

            label_idx = batch['y_true'][i]

            if not true_positive_idx[i]:
                continue

            # Get LIME explanation map
            text = ' '.join(batch['tokens.norm.premise'][i]) + ' ' + ' '.join(batch['tokens.norm.hypothesis'][i])
            premise_len = len(batch['tokens.norm.premise'][i])
            num_feature = len(batch['tokens.norm.premise'][i]) + len(batch['tokens.norm.hypothesis'][i])

            lime_exp = lime_explainer.explain_instance(text, partial(inference_attention, premise_len=premise_len), num_features = num_feature, num_samples=512, labels=[label_idx])

            lime_exp = dict(lime_exp.as_map()[label_idx])
            lime_map = [lime_exp.get(i, 0) for i in range(len(text.split(' ')))]

            lime_map = torch.tensor(lime_map)
            lime_map[lime_map < 0] = 0
            lime_map = rescale(lime_map)

            # Get human rationale
            rationale = batch['rationale.premise'][i] + batch['rationale.hypothesis'][i]

            if len(rationale) != len(lime_map):
                n_inconsistency += 1
                continue

            a_hat.append(lime_map.to(device))
            a_true.append(torch.tensor(rationale).to(device))
            #assert len(rationale) == len(lime_map), 'rationale and lime_map must have the same length'
            if WRITE_OUTPUT:
                lime_premise = lime_map[:premise_len]
                lime_hypothesis = lime_map[premise_len:]
                output_data.append({
                    'id': batch['id'][i],
                    'label': esnli_dm.LABEL_ITOS[label_idx],
                    'tokens.form.premise': batch['tokens.form.premise'][i],
                    'tokens.form.hypothesis': batch['tokens.norm.hypothesis'][i],
                    'a_lime.premise': lime_premise.tolist(),
                    'a_lime.hypothesis': lime_hypothesis.tolist(),
                })

        a_hat = torch.cat(a_hat)
        a_true = torch.cat(a_true)
        auprc.update(a_hat, a_true)
        pbar.update(1)

    auprc_value = auprc.compute().item()
    cumulative_auprc.append(auprc_value)
    print('AURPC', auprc_value)
    print('n_zero :', n_zero, ', n_inconsistency:', n_inconsistency)
    auprc.reset()

    # Write output
    if WRITE_OUTPUT:
        df_lime = pd.DataFrame(output_data)
        os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
        df_lime.to_json(path.join(MODEL_PATH, 'predictions', 'lime_map.json'))

# Compute confidential interval
ci_auprc = np.array(cumulative_auprc)
print('CI AUPRC:', str(round(ci_auprc.mean(), 3)) , '±' , str(round(1.96 * st.sem(ci_auprc), 3)))

  0%|          | 0/614 [00:00<?, ?it/s]

AURPC 0.23404116928577423
n_zero : 0 , n_inconsistency: 11
CI AUPRC: 0.234 ± nan
CPU times: user 44min 49s, sys: 45min 10s, total: 1h 30min
Wall time: 7min 28s


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [85]:
from modules import metrics
import numpy as np, scipy.stats as st

entropy = metrics.Entropy(normalize=True)
n_a_true = torch.tensor([])

for batch in esnli_dm.test_dataloader():
    for side in ['premise', 'hypothesis']:
        entropy.update(batch['a_true'][side][batch['y_true'] > 0])
        n_a_true_ins = batch['a_true'][side][batch['y_true'] > 0].sum(-1)
        n_a_true = torch.cat([n_a_true, n_a_true_ins])

print('entropy', entropy.compute())
entropy.reset()

print('#Annot:' ,str(round(n_a_true.numpy().mean(), 3)),'±', str(round(1.96 * st.sem(n_a_true.numpy()), 3)))

entropy tensor(0.9719)
#Annot: 2.667 ± 0.032


Find Confidence Interval

In [87]:
a = np.array([0.225, 0.223, 0.223])
str(round(a.mean(), 3)) + ' ± ' + str(round(1.96 * st.sem(a), 3))

'0.223 ± 0.001'

# Gradient-based

Notice : The saliency map can only be extract per row. It is not possible to extract the saliency map for the whole batch at once. Therefore the batch size must be 1.

## HateXPlain

In [6]:
from data_module.hatexplain_module import HateXPlainDM
from model_module import SingleLSTMAttentionModule
import torch
import platform
from modules.const import SpecToken

LSTM=1

# log to lstm models
DATASET = 'hatexplain'
MODEL_VERSION = 'run=0_lstm='+str(LSTM)

node = platform.node()
print('Current node:', node)

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

DATA_CACHE = path.join(ROOT, 'dataset_')
MODEL_PATH = path.join(ROOT, 'logs','lstm_attention', DATASET, MODEL_VERSION)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###############
# PREPARE DATA
###############
hatexplain_dm = HateXPlainDM(cache_path=DATA_CACHE, batch_size=1, fetch_data=True)
hatexplain_dm.prepare_data()
hatexplain_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=LSTM,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = SingleLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=hatexplain_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(hatexplain_dm.LABEL_ITOS),
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)

model.to(device)
model.load_state_dict(checkpoint['state_dict'])
#model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

Current node: grele-3.nancy.grid5000.fr
04-10-2023 09:44:41 |     INFO   hatexplain_module.py:prepare_data:76  Loaded vocab at /home/dunguyen/RUNS/dataset_/hatexplain/vocab.pt
04-10-2023 09:44:41 |     INFO   hatexplain_module.py:prepare_data:78  Vocab size: 24993
04-10-2023 09:44:44 |    DEBUG   single_lstm_attention.py:__init__:35  Initialize embedding from pretrained vector
04-10-2023 09:44:44 |    DEBUG   single_lstm_attention.py:__init__:57  concat_context: True


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


Compute Saliency map for a single instance

In [11]:
idx = 1000
post_tokens, label = hatexplain_dm.test_set.data.loc[idx, ['tokens.norm', 'label']]
text = ' '.join(post_tokens)
print('Text: ', text)
print('Label: ', label)

model.zero_grad()
token_ids = hatexplain_dm.text_transform([post_tokens]).to(device)
padding_mask = token_ids == hatexplain_dm.vocab[SpecToken.PAD]
embeddings = model.model.embedding(token_ids)
embeddings = embeddings.detach()
embeddings.requires_grad_(True)
y_hat, attention = model.model(embeddings=embeddings, mask=padding_mask)
y_hat[0, y_hat.argmax(-1)].backward()
saliency, _ = torch.max(embeddings.grad.data.abs(), dim=-1)

label_hat = hatexplain_dm.LABEL_ITOS[y_hat.argmax(-1).item()]
print('Prediction: ', label_hat)


Text:  rt usahotlips i never thought i would see the day when liberals would have so much contempt for one man their country that they would actually cheer for defend ms <number> north korea iran illegal immigrants deep state thursdaythought maga tds
Label:  offensive
Prediction:  normal


In [12]:
from modules import highlight

display(HTML('<div style="background-color:#f5f5f5>' + highlight(post_tokens, saliency[0]) + '</div>'))

Compute Taylor approximation through dataset

In [27]:
from modules import rescale
from tqdm.auto import tqdm
import pandas as pd

WRITE_OUTPUT = True
output_data = []

for batch in tqdm(hatexplain_dm.test_dataloader(), desc='Saliency'):

    # skip "normal" instances
    if batch['label'] == ['normal'] and not WRITE_OUTPUT: continue

    # extract embeddings from the tokens ids
    token_ids = batch['tokens.ids'].to(device)
    a_true = batch['a_true'].to(device)

    padding_mask = token_ids == hatexplain_dm.vocab[SpecToken.PAD]

    embeddings = model.model.embedding(token_ids)
    embeddings = embeddings.detach()
    embeddings.requires_grad_(True)

    # forward pass
    y_hat, attention = model.model(embeddings=embeddings, mask=padding_mask)

    # skip wrong predictions
    if (y_hat.argmax(-1).cpu() != batch['y_true']) and not WRITE_OUTPUT: continue

    # backward for saliency
    y_hat[0, y_hat.argmax(-1)].backward()

    # extract saliency
    saliency, _ = torch.max(embeddings.grad.data.abs(), dim=-1)
    saliency = rescale(saliency)

    # cumulate auprc
    auprc.update(saliency, a_true)
    model.model.zero_grad()

    # Write output
    if WRITE_OUTPUT:
        a_grad = saliency.tolist()
        
        for i in range(hatexplain_dm.batch_size):
            output_data.append({
                'id': batch['id'][i],
                'label': batch['label'][i],
                'tokens.form': batch['tokens.form'][i],
                'a_grad': a_grad[i],
            })

print('AURPC :', auprc.compute())
auprc.reset()

# Write output
if WRITE_OUTPUT:
    df_grad = pd.DataFrame(output_data)
    os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
    df_grad.to_json(path.join(MODEL_PATH, 'predictions', 'grad_map.json'))

Saliency:   0%|          | 0/1924 [00:00<?, ?it/s]

AURPC : tensor(0.2882, device='cuda:0')


In [28]:
df_grad.isnull().sum()

id             0
label          0
tokens.form    0
a_grad         0
dtype: int64

## YelpHat

In [15]:
from data_module.yelp_hat_module import YelpHat50DM
from model_module import SingleLSTMAttentionModule
import torch
import platform
from modules.const import SpecToken

LSTM=1

# log to lstm models
DATASET = 'yelphat50'
MODEL_VERSION = 'run=0_lstm='+str(LSTM)

node = platform.node()
print('Current node:', node)

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

DATA_CACHE = path.join(ROOT, 'dataset_')
MODEL_PATH = path.join(ROOT, 'logs','lstm_attention', DATASET, MODEL_VERSION)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###############
# PREPARE DATA
###############
yelphat_dm = YelpHat50DM(cache_path=DATA_CACHE, batch_size=1, fetch_data=False)
yelphat_dm.prepare_data()
yelphat_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=LSTM,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = SingleLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=yelphat_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(yelphat_dm.LABEL_ITOS),
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)

model.to(device)
model.load_state_dict(checkpoint['state_dict'])
#model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

Current node: grele-2.nancy.grid5000.fr
03-10-2023 14:14:31 |     INFO   yelp_hat_module.py:prepare_data:89  Loaded vocab at /home/dunguyen/RUNS/dataset_/yelp-hat/vocab_lemma_lower.pt
03-10-2023 14:14:31 |     INFO   yelp_hat_module.py:prepare_data:91  Vocab size: 7465
03-10-2023 14:14:31 |     INFO   dataset.py:__init__:186  Load dataset from /home/dunguyen/RUNS/dataset_/yelp-hat/yelp50.parquet
03-10-2023 14:14:34 |    DEBUG   single_lstm_attention.py:__init__:35  Initialize embedding from pretrained vector
03-10-2023 14:14:34 |    DEBUG   single_lstm_attention.py:__init__:57  concat_context: True
Model is in cuda:  True


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [17]:
import pandas as pd
from modules import rescale
from tqdm.auto import tqdm

WRITE_OUTPUT = True
output_data = []

for batch in tqdm(yelphat_dm.test_dataloader(), desc='Saliency'):

    # skip "normal" instances
    if batch['y_true'] == 0: continue

    # extract embeddings from the tokens ids
    token_ids = batch['tokens.ids'].to(device)
    a_true = batch['a_true'].to(device)

    padding_mask = token_ids == yelphat_dm.vocab[SpecToken.PAD]

    embeddings = model.model.embedding(token_ids)
    embeddings = embeddings.detach()
    embeddings.requires_grad_(True)

    # forward pass
    y_hat, attention = model.model(embeddings=embeddings, mask=padding_mask)

    # skip wrong predictions
    if y_hat.argmax(-1).cpu() != batch['y_true']: continue

    # backward for saliency
    y_hat[0, y_hat.argmax(-1)].backward()

    # extract saliency
    saliency, _ = torch.max(embeddings.grad.data.abs(), dim=-1)
    saliency = rescale(saliency)

    # cumulate auprc
    auprc.update(saliency, a_true)
    model.model.zero_grad()

    # Write output
    if WRITE_OUTPUT:
        a_grad = saliency.tolist()
        for i in range(yelphat_dm.batch_size):
            label = yelphat_dm.LABEL_ITOS[batch['y_true'][i]]
            output_data.append({
                'id': batch['id'][i],
                'label': label,
                'tokens.form': batch['tokens.form'][i],
                'a_grad': a_grad[i],
            })

print('AURPC :', auprc.compute())
auprc.reset()

# Write output
if WRITE_OUTPUT:
    df_grad = pd.DataFrame(output_data)
    os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
    df_grad.to_json(path.join(MODEL_PATH, 'predictions', 'grad_map.json'))

Saliency:   0%|          | 0/128 [00:00<?, ?it/s]

AURPC : tensor(0.3221, device='cuda:0')


In [18]:
df_grad

,id,label,tokens.form,a_grad
0,ham_part1(50words)_1,postive,"[Out, in, Twinsburg, for, work, and, was, n't,...","[1.0, 0.629777193069458, 0.46071264147758484, ..."
1,ham_part1(50words)_4,postive,"[Stopped, by, on, a, Sunday, around, 11, am, a...","[0.6521044969558716, 0.39220869541168213, 0.23..."
2,ham_part1(50words)_15,postive,"[I, really, really, enjoy, this, place, !, !, ...","[1.0, 0.894580602645874, 0.781977653503418, 0...."
3,ham_part1(50words)_17,postive,"[Really, good, Filipino, restaurant, ., It, 's...","[1.0, 0.8785166144371033, 0.7011961936950684, ..."
4,ham_part1(50words)_19,postive,"[The, most, delicious, steak, I, 've, ever, ha...","[1.0, 0.8883665800094604, 0.8167809844017029, ..."
5,ham_part1(50words)_20,postive,"[Excellent, breakfast, food, and, staff, !, !,...","[1.0, 0.8214786648750305, 0.7991160750389099, ..."
6,ham_part1(50words)_21,postive,"[The, burgers, are, good, ., , The, variety, ...","[1.0, 0.8456476330757141, 0.7636760473251343, ..."
7,ham_part1(50words)_22,postive,"[I, have, fond, memories, of, this, place, as,...","[0.8503952026367188, 0.7248960733413696, 0.688..."
8,ham_part1(50words)_23,postive,"[Second, time, here, and, I, will, continue, t...","[0.9466369152069092, 0.7135259509086609, 0.549..."
9,ham_part1(50words)_26,postive,"[Mmmm, ,, just, thinking, about, Romanelli, 's...","[0.818270206451416, 0.4990851581096649, 0.3473..."


## e-SNLI

In [5]:
from data_module.esnli_module import ESNLIDM
from model_module import DualLSTMAttentionModule
from modules.const import SpecToken
import torch
import platform

node = platform.node()

print('Current node:', node)

# log to lstm models
DATASET = 'esnli'
MODEL_VERSION = 'run=0_lstm=1'

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

LOG_PATH = path.join(ROOT, 'logs')
DATA_CACHE = path.join(ROOT, 'dataset_')


MODEL_PATH = path.join(ROOT, 'logs','lstm_attention',DATASET, MODEL_VERSION)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###############
# PREPARE DATA
###############
esnli_dm = ESNLIDM(cache_path=DATA_CACHE, batch_size=1)
esnli_dm.prepare_data()
esnli_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=1,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = DualLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=esnli_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(esnli_dm.LABEL_ITOS),
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)

model.to(device)
model.load_state_dict(checkpoint['state_dict'])

print('Model is in cuda: ',next(model.parameters()).is_cuda)

Current node: grele-2.nancy.grid5000.fr
03-10-2023 13:45:29 |     INFO   esnli_module.py:prepare_data:103  Loaded vocab at /home/dunguyen/RUNS/dataset_/esnli/vocab.pt
03-10-2023 13:45:29 |     INFO   esnli_module.py:prepare_data:105  Vocab size: 26578
03-10-2023 13:45:29 |  WARNING   dual_lstm_attention_module.py:__init__:53  Check if everything is ok here for archival !
03-10-2023 13:45:33 |    DEBUG   dual_lstm_attention.py:__init__:36  Initialize embedding from pretrained vector


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


In [16]:
from modules import rescale
from tqdm.auto import tqdm
import pandas as pd

WRITE_OUTPUT = True
output_data = []

model.train()
pbar = tqdm(desc='Gradient-based', total=len(esnli_dm.test_dataloader())//esnli_dm.batch_size)
for batch in esnli_dm.test_dataloader():

    y_true = batch['y_true']

    # skip "normal" instances
    if y_true == 0: continue

    # transform tokens to ids
    premise_ids = batch['tokens.ids.premise'].to(device)
    hypothesis_ids = batch['tokens.ids.hypothesis'].to(device)

    # extract embeddings from the tokens ids
    premise_embeddings = model.model.embedding(premise_ids).detach()
    hypothesis_embeddings = model.model.embedding(hypothesis_ids).detach()
    premise_embeddings.requires_grad_(True)
    hypothesis_embeddings.requires_grad_(True)

    # padding mask
    premise_padding =  premise_ids == esnli_dm.vocab[SpecToken.PAD]
    hypothesis_padding = hypothesis_ids == esnli_dm.vocab[SpecToken.PAD]

    y_hat, attention = model.model(
        premise_embeddings=premise_embeddings,
        hypothesis_embeddings=hypothesis_embeddings,
        premise_padding=premise_padding,
        hypothesis_padding=hypothesis_padding,
    )

    # skip wrong predictions
    if y_hat.argmax(-1) != y_true.to(device) : continue

    # backward for saliency
    y_hat[0, y_hat.argmax(-1)].backward()

    # extract saliency
    premise_saliency, _ = torch.max(premise_embeddings.grad.data.abs(), dim=-1)
    hypothesis_saliency, _ = torch.max(hypothesis_embeddings.grad.data.abs(), dim=-1)
    premise_saliency = rescale(premise_saliency)
    hypothesis_saliency = rescale(hypothesis_saliency)

    saliency = torch.cat([premise_saliency, hypothesis_saliency], axis=1)

    a_true = torch.cat((batch['a_true']['premise'], batch['a_true']['hypothesis']), axis=1)
    a_true = a_true.to(device)


    # cumulate auprc
    auprc.update(saliency, a_true)
    model.model.zero_grad()

    if WRITE_OUTPUT:
        a_grad = {
            'premise': premise_saliency.tolist(),
            'hypothesis': hypothesis_saliency.tolist()
        }
        for i in range(esnli_dm.batch_size):

            output_data.append({
                'id': batch['id'][i],
                'label': batch['label'][i],
                'tokens.form.premise': batch['tokens.form.premise'][i],
                'tokens.form.hypothesis': batch['tokens.form.hypothesis'][i],
                'a_grad.premise': a_grad['premise'][i],
                'a_grad.hypothesis': a_grad['hypothesis'][i],
            })

    pbar.update(1)

print('AURPC :', auprc.compute())
auprc.reset()

# Write output
if WRITE_OUTPUT:
    df_grad = pd.DataFrame(output_data)
    os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
    df_grad.to_json(path.join(MODEL_PATH, 'predictions', 'grad_map.json'))

Gradient-based:   0%|          | 0/9824 [00:00<?, ?it/s]

AURPC : tensor(0.4751, device='cuda:0')


# SHAP

During inference, to avoid to deal with `<pad>` token, we set `batch_size = 1`to avoid padding RNN

## HateXPlain

Load trained model

In [5]:
import platform

LSTM=1

# log to lstm models
DATASET = 'hatexplain'
MODEL_VERSION = 'run=0_lstm='+str(LSTM)

node = platform.node()
print('Current node:', node)

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

DATA_CACHE = path.join(ROOT, 'dataset_')
MODEL_PATH = path.join(ROOT, 'logs','lstm_attention', DATASET, MODEL_VERSION)

from data_module.hatexplain_module import HateXPlainDM
from model_module import SingleLSTMAttentionModule
import torch


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###############
# PREPARE DATA
###############
hatexplain_dm = HateXPlainDM(cache_path=DATA_CACHE, batch_size=1, fetch_data=True)
hatexplain_dm.prepare_data()
hatexplain_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=LSTM,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = SingleLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=hatexplain_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(hatexplain_dm.LABEL_ITOS),
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)

model.to(device)
model.load_state_dict(checkpoint['state_dict'])
#model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

Current node: grele-3.nancy.grid5000.fr
04-10-2023 16:59:17 |     INFO   hatexplain_module.py:prepare_data:76  Loaded vocab at /home/dunguyen/RUNS/dataset_/hatexplain/vocab.pt
04-10-2023 16:59:17 |     INFO   hatexplain_module.py:prepare_data:78  Vocab size: 24993
04-10-2023 16:59:24 |    DEBUG   single_lstm_attention.py:__init__:35  Initialize embedding from pretrained vector
04-10-2023 16:59:24 |    DEBUG   single_lstm_attention.py:__init__:57  concat_context: True


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


In [6]:
import shap
import torch
import numpy as np
from modules.const import SpecToken

# define a prediction function
def inference(texts):

    # decomment to see the generated texts
    #log.debug(f'texts = {texts}')
    texts = texts.tolist()
    tokens = [text.split(' ') for text in texts]
    #log.debug(f'tokens = {tokens}')
    token_ids = hatexplain_dm.text_transform(tokens).to(device)
    mask = token_ids == hatexplain_dm.vocab[SpecToken.PAD]
    y_hat, attention = model(token_ids, mask)
    y_hat = y_hat.softmax(-1).detach()

    return y_hat.cpu().numpy()

# build an explainer using a token masker
def whitesplit_tokenizer(text, return_offsets_mapping=False):
    tokens = text.split(' ')
    tokens = [token for token in tokens if token != '']
    token_ids = hatexplain_dm.text_transform(tokens)

    out = {"input_ids": token_ids}

    #log.debug(f'token_ids = {token_ids}')
    if return_offsets_mapping:
        token_pos = 0
        out["offset_mapping"] = []
        for token in tokens:
            out["offset_mapping"].append((token_pos, token_pos + len(token)))
            token_pos += len(token) + 1

    return out

#masker = shap.maskers.Text(r"\W", mask_token='<pad>')
masker = shap.maskers.Text(whitesplit_tokenizer, mask_token='_ ')
explainer = shap.Explainer(inference, masker, output_names=hatexplain_dm.LABEL_ITOS)


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_orde

In [7]:
from tqdm.auto import tqdm
from modules import rescale
import pandas as pd

WRITE_OUTPUT = True
output_data = []

model.eval()

y_normal = hatexplain_dm.LABEL_ITOS.index('normal')

for batch in tqdm(hatexplain_dm.test_dataloader(), desc='SHAP'):

    # skip "normal" instances
    if batch['label'] == ['normal']: continue

    # forward pass
    with torch.no_grad(): y_hat, _ = model(batch['tokens.ids'].to(device), batch['padding_mask'].to(device))

    # skip wrong predictions
    y_true = batch['y_true'].to(device)
    if y_hat.argmax(-1) != y_true: continue
    idx_class = batch['y_true']

    texts = [' '.join(t) for t in batch['tokens.form']]
    #shap_values = explainer(inference, masker, output_names=hatexplain_dm.LABEL_ITOS)
    shap_values = explainer(texts)
    shap_map = shap_values.values[0][:, idx_class] # same as ferret
    shap_map = torch.from_numpy(shap_map).unsqueeze(0)
    shap_map = rescale(shap_map)
    a_true = batch['a_true']
    auprc.update(shap_map, a_true)
    
    # Write output
    if WRITE_OUTPUT:
        a_shap = shap_map.tolist()
        for i in range(hatexplain_dm.batch_size):

            output_data.append({
                'id': batch['id'][i],
                'label': batch['label'][i],
                'tokens.form': batch['tokens.form'][i],
                'a_shap': a_shap[i],
            })
            

print('AURPC :', auprc.compute())
auprc.reset()

# Write output
if WRITE_OUTPUT:
    df_grad = pd.DataFrame(output_data)
    os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
    df_grad.to_json(path.join(MODEL_PATH, 'predictions', 'shap_map.json'))

SHAP:   0%|          | 0/1924 [00:00<?, ?it/s]

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check th

AURPC : tensor(0.5716)


In [57]:
explainer(texts)

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check th

.values =
array([[[ 4.30897649e-02, -3.68878433e-02, -6.20191451e-03],
        [ 7.59007961e-02, -1.90536332e-02, -5.68471635e-02],
        [ 1.67847860e-02, -1.72856340e-02,  5.00829890e-04],
        [-9.54214181e-03, -5.64684285e-03,  1.51889864e-02],
        [-9.10043670e-03,  1.82778924e-03,  7.27266259e-03],
        [ 7.01700444e-02, -2.15084490e-02, -4.86615822e-02],
        [ 4.46749403e-02, -8.92956613e-03, -3.57454028e-02],
        [-1.74822657e-02, -5.48543723e-03,  2.29677260e-02],
        [ 1.41731615e-02, -1.49268152e-02,  7.53642991e-04],
        [ 3.07067351e-02, -2.17437821e-02, -8.96290131e-03],
        [-7.12275479e-02, -8.30491644e-03,  7.95324370e-02],
        [-5.01201563e-01, -6.88230785e-02,  5.70024632e-01]]])

.base_values =
array([[0.44685635, 0.24647313, 0.30667046]])

.data =
(array(['<user> ', '<user> ', 'why ', 'are ', 'you ', 'repeating ',
       'yourself ', 'are ', 'you ', 'a ', 'little ', 'retarded'],
      dtype=object),)

In [62]:
shap.plots.text(shap_values)

## YelpHat

In [5]:
import platform

LSTM=1

# log to lstm models
DATASET = 'yelphat50'
MODEL_VERSION = 'run=0_lstm='+str(LSTM)

node = platform.node()
print('Current node:', node)

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

DATA_CACHE = path.join(ROOT, 'dataset')
MODEL_PATH = path.join(ROOT, 'logs','lstm_attention', DATASET, MODEL_VERSION)

from data_module.yelp_hat_module import YelpHat50DM
from model_module import SingleLSTMAttentionModule
import torch


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

###############
# PREPARE DATA
###############
yelphat_dm = YelpHat50DM(cache_path=DATA_CACHE, batch_size=1, fetch_data=True)
yelphat_dm.prepare_data()
yelphat_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=LSTM,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = SingleLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=yelphat_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(yelphat_dm.LABEL_ITOS),
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)

model.to(device)
model.load_state_dict(checkpoint['state_dict'])
#model.eval()

print('Model is in cuda: ',next(model.parameters()).is_cuda)

Current node: grele-3.nancy.grid5000.fr
04-10-2023 17:10:10 |     INFO   yelp_hat_module.py:prepare_data:89  Loaded vocab at /home/dunguyen/RUNS/dataset/yelp-hat/vocab_lemma_lower.pt
04-10-2023 17:10:10 |     INFO   yelp_hat_module.py:prepare_data:91  Vocab size: 7465
04-10-2023 17:10:10 |     INFO   dataset.py:__init__:186  Load dataset from /home/dunguyen/RUNS/dataset/yelp-hat/yelp50.parquet
04-10-2023 17:10:14 |    DEBUG   single_lstm_attention.py:__init__:35  Initialize embedding from pretrained vector
04-10-2023 17:10:14 |    DEBUG   single_lstm_attention.py:__init__:57  concat_context: True


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


In [6]:
import shap
import torch
from modules.const import SpecToken

# define a prediction function
def inference(texts):

    # decomment to see the generated texts
    #log.debug(f'texts = {texts}')
    texts = texts.tolist()
    tokens = [text.split(' ') for text in texts]
    #log.debug(f'tokens = {tokens}')
    token_ids = yelphat_dm.text_transform(tokens).to(device)
    mask = token_ids == yelphat_dm.vocab[SpecToken.PAD]
    y_hat, attention = model(token_ids, mask)
    y_hat = y_hat.softmax(-1).detach()

    return y_hat.cpu().numpy()

# build an explainer using a token masker
def whitesplit_tokenizer(text, return_offsets_mapping=False):
    tokens = text.split(' ')
    tokens = [token for token in tokens if token != '']
    token_ids = yelphat_dm.text_transform(tokens)

    out = {"input_ids": token_ids}

    #log.debug(f'token_ids = {token_ids}')
    if return_offsets_mapping:
        token_pos = 0
        out["offset_mapping"] = []
        for token in tokens:
            out["offset_mapping"].append((token_pos, token_pos + len(token)))
            token_pos += len(token) + 1

    return out

#masker = shap.maskers.Text(r"\W", mask_token='<pad>')
masker = shap.maskers.Text(whitesplit_tokenizer, mask_token='_ ')
explainer = shap.Explainer(inference, masker, output_names=yelphat_dm.LABEL_ITOS)


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_orde

In [9]:
from tqdm.auto import tqdm
from modules import rescale
import pandas as pd

WRITE_OUTPUT = True
output_data = []

model.eval()
for batch in tqdm(yelphat_dm.test_dataloader(), desc='SHAP'):

    # skip "normal" instances
    if batch['label'] == ['negative']: continue

    # forward pass
    with torch.no_grad(): y_hat, _ = model(batch['tokens.ids'].to(device), batch['padding_mask'].to(device))

    # skip wrong predictions
    y_true = batch['y_true'].to(device)
    if (y_hat.argmax(-1) != y_true) and not WRITE_OUTPUT: continue
    idx_class = batch['y_true']

    texts = [' '.join(t) for t in batch['tokens.norm']]
    #shap_values = explainer(inference, masker, output_names=hatexplain_dm.LABEL_ITOS)
    shap_values = explainer(texts)
    a_shap = shap_values.values[0][:, idx_class] # same as ferret
    a_shap = torch.from_numpy(a_shap).unsqueeze(0)
    a_shap = rescale(a_shap)
    a_true = batch['a_true']
    assert a_shap.shape == a_true.shape
    auprc.update(a_shap, a_true)
    
    # Write output
    if WRITE_OUTPUT:
        a_shap = a_shap.tolist()
        for i in range(yelphat_dm.batch_size):
            label = yelphat_dm.LABEL_ITOS[batch['y_true'][i]]
            output_data.append({
                'id': batch['id'][i],
                'label': label,
                'tokens.form': batch['tokens.form'][i],
                'a_shap': a_shap[i],
            })

print('AURPC :', auprc.compute())
auprc.reset()

# Write output
if WRITE_OUTPUT:
    df_lime = pd.DataFrame(output_data)
    df_lime.to_json(path.join(MODEL_PATH, 'predictions', 'shap_map.json'))

SHAP:   0%|          | 0/300 [00:00<?, ?it/s]

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check th

AURPC : tensor(0.3496)


## e-SNLI

In [6]:
from data_module.esnli_module import ESNLIDM
from model_module import DualLSTMAttentionModule
import torch
import platform

node = platform.node()

print('Current node:', node)

# log to lstm models
DATASET = 'esnli'
MODEL_VERSION = 'run=0_lstm=1'

if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

LOG_PATH = path.join(ROOT, 'logs')
DATA_CACHE = path.join(ROOT, 'dataset_')

MODEL_PATH = path.join(ROOT, 'logs','lstm_attention',DATASET, MODEL_VERSION)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###############
# PREPARE DATA
###############
esnli_dm = ESNLIDM(cache_path=DATA_CACHE, batch_size=1)
esnli_dm.prepare_data()
esnli_dm.setup('test')

###############
# MODEL MODULE
###############

# model's parameters (could be changed in other versions)
m_kwargs = dict(
    n_context=1,
    d_embedding=300,
    pretrained_vectors='glove.840B.300d',
)

# leave the default ones
model = DualLSTMAttentionModule(
    cache_path=ROOT,
    mode='dev',
    vocab=esnli_dm.vocab,
    concat_context=True,
    data=DATASET,
    num_class=len(esnli_dm.LABEL_ITOS),
    **m_kwargs)

ckpt_path = MODEL_PATH + '/checkpoints/best.ckpt'
checkpoint = torch.load(ckpt_path, map_location=device)

model.to(device)
model.load_state_dict(checkpoint['state_dict'])

print('Model is in cuda: ',next(model.parameters()).is_cuda)

Current node: grele-2.nancy.grid5000.fr
06-10-2023 00:13:34 |     INFO   esnli_module.py:prepare_data:103  Loaded vocab at /home/dunguyen/RUNS/dataset_/esnli/vocab.pt
06-10-2023 00:13:34 |     INFO   esnli_module.py:prepare_data:105  Vocab size: 26578
06-10-2023 00:13:35 |  WARNING   dual_lstm_attention_module.py:__init__:53  Check if everything is ok here for archival !
06-10-2023 00:13:39 |    DEBUG   dual_lstm_attention.py:__init__:36  Initialize embedding from pretrained vector


/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Model is in cuda:  True


In [7]:
import shap
import torch
from modules.const import SpecToken

# define a prediction function
def inference(texts):

    # decomment to see the generated texts
    #log.debug(f'texts = {texts}')
    batch_tokens = [text.split(' ') for text in texts]

    tokens_premise = []
    tokens_hypothesis = []
    for tokens in batch_tokens:
        if len(tokens) == 1:
            # Special case, where input text is '_'
            tokens_premise.append(tokens)
            tokens_hypothesis.append(tokens)
        else:
            sep_index = tokens.index(SpecToken.SEP.value) if SpecToken.SEP.value in tokens else -1
            tokens_premise.append(tokens[:sep_index])
            tokens_hypothesis.append(tokens[sep_index+1:])

    ids_premise = esnli_dm.text_transform(tokens_premise)
    ids_hypothesis = esnli_dm.text_transform(tokens_hypothesis)
    padding_mask_premise = ids_premise == esnli_dm.vocab[SpecToken.PAD]
    padding_mask_hypothesis = ids_hypothesis == esnli_dm.vocab[SpecToken.PAD]

    y_hat, attention = model(
        premise_ids=ids_premise.to(device),
        hypothesis_ids=ids_hypothesis.to(device),
        premise_padding=padding_mask_premise.to(device),
        hypothesis_padding=padding_mask_hypothesis.to(device),
    )
    y_hat = y_hat.softmax(-1).detach()

    return y_hat.cpu().numpy()

# build an explainer using a token masker
def whitesplit_tokenizer(text, return_offsets_mapping=False):

    tokens = text.split(' ')
    tokens = [token for token in tokens if token != '']# remove empty tokens # remove empty tokens

    input_ids = esnli_dm.text_transform(tokens)

    out = {"input_ids": input_ids}

    if return_offsets_mapping:
        token_pos = 0
        out["offset_mapping"] = []
        for token in tokens:
            out["offset_mapping"].append((token_pos, token_pos + len(token)))
            token_pos += len(token) + 1

    return out

#masker = shap.maskers.Text(r"\W", mask_token='<pad>')
masker = shap.maskers.Text(whitesplit_tokenizer, mask_token='_ ')
explainer = shap.Explainer(inference, masker, output_names=esnli_dm.LABEL_ITOS)

/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/dunguyen/virtualenv/eps/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_orde

In [12]:
esnli_dm.test_set.data.rename(columns={'premise_heuristic': 'heuristic.premise', 'hypothesis_heuristic': 'heuristic.hypothesis'}, inplace=True)

In [15]:
esnli_dm.test_set.data.to_parquet(esnli_dm.test_set.parquet_path)

In [7]:
from tqdm.auto import tqdm
from modules import rescale
import pandas as pd

model.eval()

WRITE_OUTPUT = True
output_data = []

for batch in tqdm(esnli_dm.test_dataloader(), desc='SHAP'):

    # skip "normal" instances
    if batch['label'] == ['neutral']: continue

    # forward pass
    with torch.no_grad():
        y_hat, a_hat = model(
            premise_ids=batch['tokens.ids.premise'].to(device),
            hypothesis_ids=batch['tokens.ids.hypothesis'].to(device),
            premise_padding=batch['padding_mask']['premise'].to(device),
            hypothesis_padding=batch['padding_mask']['hypothesis'].to(device),
        )

    # skip wrong predictions
    y_true = batch['y_true'].to(device)
    if (y_hat.argmax(-1) != y_true) and not WRITE_OUTPUT: continue
    idx_class = batch['y_true']

    batch_text = []
    for sent_p, sent_h in zip(batch['tokens.norm.premise'], batch['tokens.norm.hypothesis']):
        text = ' '.join(sent_p + [SpecToken.SEP] + sent_h)
        batch_text.append(text)

    #shap_values = explainer(inference, masker, output_names=hatexplain_dm.LABEL_ITOS)
    shap_values = explainer(batch_text)
    shap_map = shap_values.values[0][:, idx_class] # same as ferret
    shap_map = torch.from_numpy(shap_map).unsqueeze(0)
    shap_map = rescale(shap_map)
    a_true = batch['a_true']
    batch_size = a_true['premise'].shape[0]
    tensor_sep = torch.zeros((batch_size, 1))
    flat_a_true = torch.cat((a_true['premise'], tensor_sep, a_true['hypothesis']), dim=1)
    auprc.update(shap_map, flat_a_true)

    if shap_map.shape != flat_a_true.shape:
        log.debug(f'pairID = {batch["id"]}')
        log.debug(f"ids len = {batch['tokens.ids.premise'].shape}, {batch['tokens.ids.hypothesis'].shape}")
        log.debug(f"norm len = {len(batch['tokens.norm.premise'])}, {len(batch['tokens.norm.hypothesis'])}")
        log.debug(f"batch_text = {batch_text}")

    #log.info('AUPRC: {}'.format(auprc.compute()))

    if WRITE_OUTPUT:

        shap_map = shap_map.tolist()

        for i in range(esnli_dm.batch_size):

            premise_len = len(batch['tokens.form.premise'][i])
            a_shap_premise = shap_map[i][:premise_len]
            a_shap_hypothesis = shap_map[i][premise_len+1:]
            assert len(a_shap_hypothesis) == len(batch['tokens.form.hypothesis'][i])

            output_data.append({
                'id': batch['id'][i],
                'label': batch['label'][i],
                'tokens.form.premise': batch['tokens.form.premise'][i],
                'tokens.form.hypothesis': batch['tokens.form.hypothesis'][i],
                'a_shap.premise': a_shap_premise,
                'a_shap.hypothesis': a_shap_hypothesis,
            })

print('AURPC :', auprc.compute())
auprc.reset()

# Write output
if WRITE_OUTPUT:
    df_shap = pd.DataFrame(output_data)
    os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
    df_shap.to_json(path.join(MODEL_PATH, 'predictions', 'shap_map.json'))

SHAP:   0%|          | 0/9824 [00:00<?, ?it/s]

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check th

AURPC : tensor(0.2425)


NameError: name 'pd' is not defined

In [9]:
df_grad = pd.DataFrame(output_data)
os.makedirs(path.join(MODEL_PATH, 'predictions'), exist_ok=True)
df_grad.to_json(path.join(MODEL_PATH, 'predictions', 'shap_map.json'))

In [11]:
path.join(MODEL_PATH, 'predictions', 'shap_map.json')

'/home/dunguyen/RUNS/logs/lstm_attention/esnli/run=0_lstm=1/predictions/shap_map.json'